In [8]:
# ✅ Step 1: Upgrade pip, setuptools, and wheel first (important!)
%pip install --upgrade pip setuptools wheel

# ✅ Step 2: Install all required libraries (with fixed package names)
%pip install \
    "sentence-transformers>=2.2.2,<3.0" \
    langchain \
    flask \
    pypdf \
    python-dotenv \
    "pinecone-client[grpc]" \
    langchain-pinecone \
    langchain-community \
    langchain-openai \
    langchain-experimental


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install pymupdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
%pwd

'd:\\Introduction to Programming with Python\\PythonProjects'

In [ ]:
import os 
os.chdir("../")


In [29]:
%pwd

'd:\\Introduction to Programming with Python\\PythonProjects'

In [30]:
%cd "D:/Introduction to Programming with Python/PythonProjects/Medical chat Bot"


D:\Introduction to Programming with Python\PythonProjects\Medical chat Bot


C:\Users\Muhammad Asim\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [31]:
%pwd


'D:\\Introduction to Programming with Python\\PythonProjects\\Medical chat Bot'

In [32]:
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyMuPDFLoader
                             )
    document = loader.load()
    return document

In [34]:
%pwd

'D:\\Introduction to Programming with Python\\PythonProjects\\Medical chat Bot'

In [35]:
extracted_data = load_pdf(data='Data/')

In [11]:
# extracted_data

In [36]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [37]:
text_chunks = text_split(extracted_data)
print("Length of Text chunks", len(text_chunks))

Length of Text chunks 5777


In [12]:
# text_chunks

In [38]:
from langchain.embeddings import HuggingFaceEmbeddings

In [39]:
def download_hugging_face_embeddings():
    # Download the Hugging Face Embeddings
    emdeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return emdeddings

In [40]:
embeddings = download_hugging_face_embeddings()

C:\Users\Muhammad Asim\AppData\Local\Temp\ipykernel_3484\2436550048.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emdeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


In [41]:
query_result = embeddings.embed_query("Hello World")
print("legth" , len(query_result))

legth 384


In [17]:
# query_result

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
import os

PINECONE_API_KEY = "pcsk_tcVhe_UNxCvkWUa1d5fTpKNjqp5aq7FtRC62y44Ek3Rs53wJxN2WeborV6PaSm8vVd44S"
PINECONE_ENVIRONMENT = "us-east-1"

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_ENVIRONMENT"] = PINECONE_ENVIRONMENT


In [20]:
# PINE_CONE_API_KEY = os.environ.get('PINE_CONE_API_KEY')

In [21]:
# PINE_CONE_API_KEY

In [22]:
# from pinecone.grpc import PineconeGRPC as pinecone
# from pinecone import ServerlessSpec

# pc = pinecone(api_key= PINE_CONE_API_KEY)
# index_name = "medicalchatbot"

# pc.create_index(
#     name= index_name,
#     dimension= 384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [44]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec

pc = pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalchatbot"

if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,  # this must match your embedding output dim
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


In [24]:
# import os
# os.environ["PINE_CONE_API_KEY"] = PINE_CONE_API_KEY

In [ ]:
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings,
#     pinecone_api_key=PINE_CONE_API_KEY,
#     pinecone_environment="us-east-1"  # must match the one in ServerlessSpec
# )


In [45]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)


In [46]:
from langchain_pinecone import PineconeSparseVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [47]:
docsearch

In [48]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [49]:
retrived_docs = retriver.invoke("what is correlation?")

In [50]:
retrived_docs

[Document(id='566893a6-71cd-4bfb-9c5c-477efa12f492', metadata={'author': '', 'creationDate': "D:20041218170002-05'00'", 'creationdate': '2004-12-18T17:00:02-05:00', 'creator': '', 'file_path': 'Data\\Medical_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20041218161531-06'00'", 'moddate': '2004-12-18T16:15:31-06:00', 'page': 333.0, 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'subject': '', 'title': '', 'total_pages': 637.0, 'trapped': ''}, page_content='• Repression. The person pushes anxious thoughts or\nideas out of conscious awareness.\n• Displacement. Anxiety from one source is attached to a\ndifferent object or event. Phobias are an example of the\nmechanism of displacement in psychoanalytic theory.\n• Rationalization. The person justifies the anxious feel-\nings by saying that any normal person would feel anx-\nious in their situation.\n• Somatization. The anxiety emerges in the form of phys-\nical complaints and illnesses, such as r

In [58]:
%pip install langchain langchain-community openai


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [64]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "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"

llm = ChatOpenAI(
    model="deepseek/deepseek-r1-0528:free",  
    base_url="https://openrouter.ai/api/v1",  
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.4 ,
    max_tokens= 500
)


In [65]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are my assistant for Q/A tasks"
    "User the following Piece of Retrived Context to answer the question at the end."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer"
    "Use three Sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"


)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)



In [66]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [70]:
response = rag_chain.invoke({"input": "What is Acne ?"})
print(response["answer"])

Based on the provided context:

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil (sebum), dead skin cells, and bacteria, leading to inflammation. This blockage prevents sebum from flowing out normally, causing small swellings on the skin surface.
